In [23]:
#Power BI Export

import importlib.util
spec = importlib.util.spec_from_file_location("MongoTwitter.models", r"C:\Users\victor.santos2\OneDrive - Grupo Marista\TCC\Twitter Sentiment Analysis\MongoTwitter\models.py")
models = importlib.util.module_from_spec(spec)
spec.loader.exec_module(models)
import pandas as pd
import mongoengine

mongoengine.register_connection(alias='core',name='sentimentAnalysis')
#1


In [24]:
candidates = [('lhmandetta',2726301380,'Mandetta'),
('EduardoLeite_',123891446,'Eduardo Leite'),
('DatenaOficial',68722955, 'Datena'),
('cirogomes',33374761, 'Ciro Gomes'),
('jdoriajr',64706049, 'Dória'),
('LulaOficial',2670726740,'Lula'),
('jairbolsonaro',128372940, 'Bolsonaro'),]
can_df = pd.DataFrame(data=candidates, columns=['Handle', 'author_id','Name'])
#2


In [25]:
objs_tweets = models.Tweets.objects().all().values_list('id', 'tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count')
tweets = pd.DataFrame.from_records(data=objs_tweets, columns=['id','tweet_id','author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
tweets_df = tweets
tweets = tweets.drop(columns=['id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
tweets = tweets.rename(columns={"tweet_id": "conversation_id"})
tweets = tweets.merge(can_df)
tweets = tweets.drop(columns=['author_id'])
#3

In [26]:
tweets_sentiments = models.TweetsSentiment.objects.all().values_list('tweet_id','length','sentiment', 'negative','positive', 'neutral','clean_text')
tweets_sentiments_df = pd.DataFrame.from_records(data=tweets_sentiments, columns=['tweet_id','length','sentiment', 'negative','positive', 'neutral','clean_text'])
tweets_df = tweets_df.merge(tweets_sentiments_df)
#4


In [27]:
objs_replies = models.Replies.objects.all().values_list('id','reply_id', 'in_reply_to_user_id', 'conversation_id' ,'author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count')
replies_df = pd.DataFrame.from_records(data=objs_replies, columns=['id','reply_id', 'in_reply_to_user_id', 'conversation_id' ,'author_id','created_at','text','lang','retweet_count','reply_count','like_count','quote_count'])
#5


In [28]:
replies_sentiments = models.RepliesSentiment.objects.all().values_list('reply_id','length','sentiment', 'negative','positive', 'neutral','clean_text')
replies_sentiments_df = pd.DataFrame.from_records(data=replies_sentiments, columns=['reply_id','length','sentiment', 'negative','positive', 'neutral','clean_text'])
replies_df = replies_df.merge(replies_sentiments_df)
#6

In [30]:
replies_df = replies_df.merge(tweets)
tweets_df = tweets_df.merge(can_df)
#7
tweets_df

,id,tweet_id,author_id,created_at,text,lang,retweet_count,reply_count,like_count,quote_count,length,sentiment,negative,positive,neutral,clean_text,Handle,Name
0,613ec5589eb2bd97646d86ad,1437158653623160837,2670726740,2021-09-12 20:58:18,@dilmabr 🙏🏼,und,123,76,6411,7,3,neutral,0.05,0.09,0.86,🙏🏼,LulaOficial,Lula
1,613ec5599eb2bd97646d86ae,1437054980884684804,2670726740,2021-09-12 14:06:21,Recomendação de leitura para o domingo: Amor à...,pt,1812,1529,18236,271,104,positive,0.00,1.00,0.00,recomendação de leitura para o domingo: amor à...,LulaOficial,Lula
2,613ec5599eb2bd97646d86af,1436397090003881984,2670726740,2021-09-10 18:32:07,"“Olho pra trás, vejo a estrada que eu trilhei,...",pt,783,538,7294,84,125,negative,0.62,0.02,0.36,"“olho pra trás, vejo a estrada que eu trilhei,...",LulaOficial,Lula
3,613ec5599eb2bd97646d86b0,1436156966817181698,2670726740,2021-09-10 02:37:58,"@leandraleal Feliz Aniversário, companheira! 💫",pt,87,84,7371,7,32,positive,0.00,0.99,0.01,"feliz aniversário, companheira!",LulaOficial,Lula
4,613ec55a9eb2bd97646d86b1,1435999486162186244,2670726740,2021-09-09 16:12:11,@MussumAlive @manobrown 👊🏼,und,10,15,636,1,4,neutral,0.05,0.09,0.86,👊🏼,LulaOficial,Lula
5,613ec55a9eb2bd97646d86b2,1435966262656471041,2670726740,2021-09-09 14:00:10,Já ouviram? https://t.co/7tDefUtuPq,pt,629,510,10940,52,11,neutral,0.01,0.02,0.97,já ouviram?,LulaOficial,Lula
6,613ec55b9eb2bd97646d86b3,1435913362689437700,2670726740,2021-09-09 10:29:58,Bom dia. Quem já ouviu o #ManoaMano?! Papo com...,pt,1622,530,12676,259,8,positive,0.01,0.87,0.12,bom dia.,LulaOficial,Lula
7,613ec55b9eb2bd97646d86b4,1435809504571150338,2670726740,2021-09-09 03:37:16,"Boa noite! Tá no ar… Satisfação, @manobrown 👊🏼...",pt,862,208,12197,61,10,positive,0.01,0.98,0.01,boa noite!,LulaOficial,Lula
8,613ec55c9eb2bd97646d86b5,1435697208352034817,2670726740,2021-09-08 20:11:03,"Hoje, meia noite. 👊🏼 https://t.co/M9qMpkgRyo",pt,1436,347,17212,220,17,neutral,0.03,0.05,0.92,"hoje, meia noite.",LulaOficial,Lula
9,613ec55c9eb2bd97646d86b6,1435696133796413447,2670726740,2021-09-08 20:06:46,@jefdelgado @manobrown uma camisa bonita dessas,pt,10,11,437,6,26,positive,0.00,0.99,0.01,uma camisa bonita dessas,LulaOficial,Lula
